_Note préliminaire :_ La dernière session d'entraînement a été exécutée avec ce notebook sur une instance avec un GPU L4. Le notebook est cependant conçu pour être éxécuté avec l'environnement jupyter-pytorch-gpu du SSP Cloud, et c'est d'ailleurs avec cet environnement que nous avons fait la plupart des tests. De nombreuses dépendances y sont présentes et ne sont pas explicitement installées dans ce notebook. La cellule ci-dessous installe les dépendances supplémentaires. Du fait de l'installation de SentencePiece, il faut **impérativement redémarrer** le kernel jupyter après.

In [1]:
!pip install pandas numpy html2text torch transformers tqdm SentencePiece pyarrow


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Entrainement et évaluation du modèle

Ce notebook détaille l'entraînement du modèle. On travaille avec le jeu de données des liens-perturbations objets. L'objectif est d'entraîner un modèle pour prédire 
la durée de l'incident à partir du message d'alerte, tout en utilisant les données catégoriques supplémentaires :

- la cause (colonne `cause`), ayant trois modalités : `PERTURBATION`, `TRAVAUX`, et `INFORMATION`
- le niveau d'incident (colonne `severity`) ayant trois modalités : `PERTURBEE`, `BLOQUANTE`, et `INFORMATION`
- le type d'object impacté (colonne `object_type`) ayant quatre modalités : `line`, `stop_point`, `network`, et `stop_area`
- le mode de transport (colonne `line_mode`) ayant six modalités : `Bus`, `Tramway`, `Metro`, `RapidTransit`, `LocalTrain`, et `Funicular`

Pour cela, notre solution finale (après plusieurs itérations) consiste à utiliser un modèle d'embedding spécialisé en 
français, _CamemBERT_ (voir le [papier](https://arxiv.org/abs/1911.03894) associé). Nous utilisons précisément la 
version pré-entrainée `camembert-base` (voir la 
[carte du modèle sur HuggingFace](https://huggingface.co/almanach/camembert-base)), de 110 millions de paramètres. 
Elle est entraînée sur le corpus OSCAR, jeu de 138 GO de données générales en français, jeu qui ressemble plus à notre
situation que celui de _FlauBERT_ (voir [carte HuggingFace](https://huggingface.co/flaubert/flaubert_base_uncased)), 
autre modèle d'embedding spécialisé en français que nous aurions pu utiliser.

Ce modèle est basé sur l'architecture _RoBERTa_ (voir le [papier](https://arxiv.org/pdf/1907.11692) associé), une 
itération un peu plus moderne de _BERT_ (voir le [papier](https://arxiv.org/abs/1810.04805) associé), et calcule donc 
un embedding du message d'alerte. Nous le combinons avec une tête de régression assez simple, à savoir un perceptron 
multi-couche assez simple (simplement deux-couches et une ReLU), chargée de prédire en même temps la durée de la 
perturbation, mais aussi les données catégoriques associées. Cette approche permet donc (en théorie) de faire 
apprendre au modèle à reconnaître les différents types d'incidents, en plus de prédire leur durée.

Pour cela, on ne garde que les incidences se déroulant sur un jour au maximum (c'est principalement ce qui 
nous intéresse), les analyses préliminaires ayant montré qu'il pouvait sinon y avoir beaucoup de valeurs préliminaires. 
La durée est représentée sous deux catégories flottantes (heures et minutes), traitées en amont avec une normalisation 
robuste, et évaluées par une erreur quadratique moyenne. Les données catégoriques sont représentées avec un encodage one-hot, et évaluées par une perte d'entropie croisée binaire. Le modèle est entraînée avec une perte correspondant à la somme de ces deux métriques, et l'optimisateur AdamW (Adam étant utilisé par CamemBERT et RoBERTa, voir les papiers associés, et AdamW semblant en être une version légèrement supérieure).

Le diagramme ci-dessous résume le modèle :

![diagramme récapitulatif](images/modele.png)

### Traitement des données

On commence par importer le jeu de données des liens objets-perturbations, générées préalablement.

In [2]:
import pandas as pd

df_disruptions = pd.read_feather("data/objects_disruptions.feather")

Le champ `message` issu de l'api est formaté en HTML et non pas en texte brut. Avec le module `html2text`, on le convertit donc en texte plein et on l'assemble dans une colonne avec la colonne `title` (titre du message). 

In [3]:
from html2text import html2text

# Prétraitement des données
df_disruptions["text"] = df_disruptions["title"] + "\n\n" + df_disruptions["message"].apply(html2text)

On convertit les colonnes `begin` et `end` (début et fin de la perturbation) de leur champ texte aux formats `datetime[*]` adaptés. On calcule la différence dans une colonne `delta` (format `timedelta[*]`), qu'on sépare en trois colonnes `duration_days`, `duration_hours` et `duration_minutes` correspondant respectivement à la durée de l'incident en jours, en heures et minutes.

In [4]:
# les colonnes begin et end sont au format YYYYMMDDThhmmss donc on les parse avec le format %Y%m%dT%H%M%S.
# Il ne devrait pas y avoir d'exception, mais dans le doute errors="coerce" permet d'avoir des NaN.
df_disruptions["begin"] = pd.to_datetime(df_disruptions["begin"], format="%Y%m%dT%H%M%S", errors="coerce")
df_disruptions["end"] = pd.to_datetime(df_disruptions["end"], format="%Y%m%dT%H%M%S", errors="coerce")
df_disruptions["delta"] = df_disruptions["end"] - df_disruptions["begin"]

# Calcul des jours, heures et minutes à partir de la colonne delta
df_disruptions["duration_days"] = df_disruptions["delta"].dt.days
df_disruptions["duration_hours"] = df_disruptions["delta"].dt.seconds // 3600  # Heures restantes
df_disruptions["duration_minutes"] = (df_disruptions["delta"].dt.seconds % 3600) // 60  # Minutes restantes

On peut alors vérifier la distribution des durées en jour en regardant celle de `duration_days` :

In [5]:
df_disruptions["duration_days"].quantile([n/10 for n in range(1, 11)])

0.1       0.0
0.2       0.0
0.3       0.0
0.4       0.0
0.5       0.0
0.6       0.0
0.7       0.0
0.8       0.0
0.9      23.0
1.0    5113.0
Name: duration_days, dtype: float64

On observe que plus de $80 \%$ des pertubations se déroulent sur moins d'un jour. On filtre donc dans un nouveau dataframe `df_disruptions_filtered` ces perturbations :

In [6]:
df_disruptions_filtered = df_disruptions[df_disruptions["duration_days"] < 1]

On va ensuite appliquer une normalisation robuste des colonnes `duration_hours` et `duration_hours` sur le jeu qu'on vient de filtrer. On la calcule avec

$$ X_{i,\text{robuste}} = \dfrac{X_i - X_{q,50}}{X_{q,75} - X_{q, 25}} $$

où $X_{q,25}, X_{q,50}, X_{q,75}$ sont le premier, deuxième (médianne) et troisième quartile de $X$ (il existe des versions où on soustrait plutôt le premier quartile). Cette normalisation permet de tenir compte des valeurs aberrantes.  

In [7]:
# Calcul des quartiles
quantiles_hours = df_disruptions_filtered["duration_hours"].quantile([0.25, 0.5, 0.75])
quantiles_minutes = df_disruptions_filtered["duration_minutes"].quantile([0.25, 0.5, 0.75])

# Calcul de l'écart inter-quartile
iqr_hours = quantiles_hours[0.75] - quantiles_hours[0.25]
iqr_minutes = quantiles_minutes[0.75] - quantiles_minutes[0.25]

# Normalisation
df_disruptions_filtered["hours"] = (df_disruptions_filtered["duration_hours"] - quantiles_hours[0.5]) / iqr_hours
df_disruptions_filtered["minutes"] = (df_disruptions_filtered["duration_minutes"] - quantiles_minutes[0.5]) / iqr_minutes

/tmp/ipykernel_710/496045166.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_disruptions_filtered["hours"] = (df_disruptions_filtered["duration_hours"] - quantiles_hours[0.5]) / iqr_hours
/tmp/ipykernel_710/496045166.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_disruptions_filtered["minutes"] = (df_disruptions_filtered["duration_minutes"] - quantiles_minutes[0.5]) / iqr_minutes


On calcule ensuite un DataFrame `df` final, avec l'encodage one-hot des colonnes `cause`, `severity`, `object_type`, `line_mode`. On enregistre dans `targets` les colonnes des cibles, et on convertit ces colonnes au format flottant. 

In [8]:
# One-hot encoding des colonnes catégoriques
df = pd.get_dummies(
    df_disruptions_filtered[["text", "hours", "minutes", "cause", "severity", "object_type", "line_mode"]],
    columns=["cause", "severity", "object_type", "line_mode"]
)

# On note les colonnes des cibles
targets = [k for k in df.columns if k != "text"]

# Conversion des colonnes cibles en flottants
for col in targets[1:]:
    df[col] = df[col].astype(float)

On sépare ce DataFrame en un jeu d'entraînement (80%) et de test (20%), en l'ayant mélangé. Par soucis de reproductibilité, le `random_state` a été fixé à `123456789`. On sépare les données d'entrées (nommées $X$ par convention), qui sont la colonne `text` et les cibles (nommées $y$ par convention). 

In [9]:
train_size = 0.8

# Diviser le DataFrame 
df_shuffled = df.sample(frac=1, random_state=123456789).reset_index(drop=True)  # Mélanger
train_df = df_shuffled.iloc[:int(len(df) * train_size)]
test_df = df_shuffled.iloc[int(len(df) * train_size):]

# Texte (entrée) et cibles
X_train = train_df["text"].values.tolist()
X_test = test_df["text"].values.tolist()
y_train = train_df[targets].values
y_test = test_df[targets].values

On va ensuite tokeniser les messages en utilisant le tokenizer de CamemBERT. Voir la [documentation](https://huggingface.co/docs/transformers/en/model_doc/camembert#transformers.CamembertTokenizer).

In [10]:
from transformers import CamembertTokenizer
# Chargement le tokenizer
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

# Tokenisation des X d'entraînement et de test
def tokenize_texts(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

train_encodings = tokenize_texts(X_train)
test_encodings = tokenize_texts(X_test)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Entraînement et évaluation

Pour l'entraînement et l'évaluation, on utilise le module `Pytorch` et les utilitaires adaptés. On crée donc d'abord un objet `RegressionDataset` héritant du `Dataset` pytorch, qu'on instancie pour le jeu d'entraînement et d'évaluation.

In [11]:
import torch
from torch.utils.data import Dataset

class RegressionDataset(Dataset):
    def __init__(self, encodings, labels):
        """
        Initialise un Dataset torch, avec deux atributs
        :param encodings: encodings issus du tokenizer CamemBERT.
        :param labels: cibles (ici multiples, flotants pour la durée et les cibles booléenes)
        """
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        """
        Méthode classique de torch, on formatte un objet pour notre cas d'utilisation.
        """
        return {
            key: val[idx] for key, val in self.encodings.items()
        } | {"labels": torch.tensor(self.labels[idx], dtype=torch.float32)}

train_dataset = RegressionDataset(train_encodings, y_train)
test_dataset = RegressionDataset(test_encodings, y_test)

On utilise des `DataLoader` pour charger progressivement ces datasets lors de l'entraînement et de l'évaluation. Le `batch_size` peut être modifié (les valeurs jusqu'à 128 semblent fonctionner sur le SSP Cloud), mais ne change visiblement pas la vitesse d'entraînement.

In [12]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

On crée un objet `CamembertForRegression`, qu'on instancie, correspondant au modèle décrit plus haut. Pour rappel :

![modele](images/modele.png)

**NB :** Le modèle est chargé directement sur CUDA. Si on lance l'entraînement sur cpu, il faudrait faire différement (par exemple utiliser la variable device plus bas).

In [13]:
from transformers import CamembertModel
import torch.nn as nn


class CamembertForRegression(nn.Module):
    def __init__(self, num_outputs):
        """
        Initialise le modèle CamemBERT pour la régression.
        :param num_outputs: Nombre de sorties de la régression
        """
        super(CamembertForRegression, self).__init__() 
        
        # Le modèle est basé sur une version pré-entraînée de CamemBERT,
        # en l'ocurrence ici camembert-base (110 millions de paramètres, entraîné sur OSCAR),
        # voir https://huggingface.co/almanach/camembert-base. On pourrait sinon
        # utiliser camembert-large (335 millions de paramètres).
        self.model = CamembertModel.from_pretrained("camembert-base")
        
        # On ajoute un perceptron multi-couche comme tête de régression, laquelle transforme
        # les embeddings produits par CamemBERT (de taille self.model.config.hidden_size)
        # en valeurs pour la tâche de régression (de taille num_outputs)
        self.regression_head = nn.Sequential(
            nn.Linear(self.model.config.hidden_size, 128),
            nn.ReLU(),
            nn.Linear(128, num_outputs)
        )

    def forward(self, input_ids, attention_mask):
        """
        Définition de la propagation avant (forward).
        :param input_ids: Identifiants des tokens (token IDs) du texte.
        :param attention_mask: Masque pour ignorer les positions de padding.
        :return: Les prédictions issues de la tête de régression.
        """
        # Passe les données dans le modèle CamemBERT
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Récupère l'embedding du token [CLS] censé être l'embedding résumé du texte d'entrée
        # voir https://arxiv.org/abs/1810.04805
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        
        # Applique la tête de régression pour produire les prédictions finales
        return self.regression_head(cls_embedding)


# Instancie le modèle
model = CamembertForRegression(num_outputs=y_train.shape[1]).to("cuda")

On instancie un optimisateur AdamW. Le taux d'apprentisage choisi ($3 \cdot 10^{-5}$) semble bien fonctionner sur quelques tests.

In [14]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=3e-5)

On crée un objet `MultiTaskLoss` correspondant à la fonction de perte décrit plus haut.

In [15]:
class MultiTaskLoss(nn.Module):
    def __init__(self):
        super(MultiTaskLoss, self).__init__()
        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, predictions, labels):
        # Séparer les cibles continues et binaires
        duration_preds = predictions[:, :3]  # Les trois premières colonnes : jours, heures, minutes
        duration_labels = labels[:, :3]

        binary_preds = predictions[:, 3:]  # Autres cibles booléennes
        binary_labels = labels[:, 3:]

        # Calcul des pertes
        mse_loss = self.mse(duration_preds, duration_labels)
        bce_loss = self.bce(binary_preds, binary_labels)

        # Retourner les pertes sous forme de scalaires
        return mse_loss.item(), bce_loss.item(), (mse_loss + bce_loss).item()
    
loss_fn = MultiTaskLoss()

Enfin, on lance l'entraînement sur 7 epochs. Pour chaque epoch, on calcule la perte d'entraînement et d'évaluation, ainsi que l'erreur quadratique moyenne sur les minutes et les secondes.

In [16]:
from tqdm import tqdm

device = "cuda" #torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(7):
    model.train()
    train_loss = 0
    train_mse_loss = 0
    train_bce_loss = 0

    for batch in tqdm(train_loader, "Entraînement"):
        optimizer.zero_grad() #On initialise l'optimisateur
    
        # on envoie les données sur CUDA
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        # on infère avec le modèle    
        predictions = model(input_ids, attention_mask)
        
        # on calcule la perte
        mse_loss, bce_loss, loss = loss_fn(predictions, labels)
    
        # on converti en tenseur
        loss_tensor = torch.tensor(loss, requires_grad=True).to(device)
        
        # rétropropagation
        loss_tensor.backward()
        
        # cliping des gradients (nous n'avons pas testé d'autres valeurs que 1 pour max_norm, par manque de temps)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
        # mise à jour des poids
        optimizer.step()
        
        # incrémentation des pertes pour l'affichage sur l'epoch    
        train_loss += loss
        train_mse_loss += mse_loss
        train_bce_loss += bce_loss

    print(f"Epoch {epoch + 1}, Training Loss: {train_loss / len(train_loader):.4f}")
    print(f"Epoch {epoch + 1}, Training MSE Loss: {train_mse_loss / len(train_loader):.4f}")
    print(f"Epoch {epoch + 1}, Training BCE Loss: {train_bce_loss / len(train_loader):.4f}")

    # enregistrement des poids du modèle à l'epoch
    torch.save(model.state_dict(), "checkpoint/epoch_" + str(epoch + 1) + ".pt")

    # Evaluation. Le code est sensiblement le même.
    model.eval()
    val_loss = 0
    val_mse_loss = 0
    val_bce_loss = 0

    # Variables pour accumuler les cibles et prédictions
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            predictions = model(input_ids, attention_mask)
            mse_loss, bce_loss, loss = loss_fn(predictions, labels)

            val_loss += loss
            val_mse_loss += mse_loss
            val_bce_loss += bce_loss

            # stockages les cibles et prédictions pour les MSE séparées
            all_labels.append(labels[:, :2])  # Garder uniquement heures et minutes
            all_predictions.append(predictions[:, :2])

    # Concatenation des résultats sur tous les batches
    all_labels = torch.cat(all_labels, dim=0)
    all_predictions = torch.cat(all_predictions, dim=0)

    # Calcul des MSE pour chaque cible
    mse_hours = torch.mean((all_labels[:, 0] - all_predictions[:, 0]) ** 2).item()
    mse_minutes = torch.mean((all_labels[:, 1] - all_predictions[:, 1]) ** 2).item()

    # Affichage des résultats
    print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(test_loader):.4f}")
    print(f"Epoch {epoch + 1}, Validation MSE Loss: {val_mse_loss / len(test_loader):.4f}")
    print(f"Epoch {epoch + 1}, Validation BCE Loss: {val_bce_loss / len(test_loader):.4f}")
    print(f"Epoch {epoch + 1}, MSE Hours: {mse_hours:.4f}")
    print(f"Epoch {epoch + 1}, MSE Minutes: {mse_minutes:.4f}")

Entraînement: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3785/3785 [14:39<00:00,  4.30it/s]


Epoch 1, Training Loss: 1.0745
Epoch 1, Training MSE Loss: 0.3734
Epoch 1, Training BCE Loss: 0.7012


Evaluation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 947/947 [03:24<00:00,  4.62it/s]


Epoch 1, Validation Loss: 1.0743
Epoch 1, Validation MSE Loss: 0.3739
Epoch 1, Validation BCE Loss: 0.7004
Epoch 1, MSE Hours: 0.7226
Epoch 1, MSE Minutes: 0.3974


Entraînement: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3785/3785 [14:39<00:00,  4.30it/s]


...

Epoch 5, Training Loss: 1.0746
Epoch 5, Training MSE Loss: 0.3734
Epoch 5, Training BCE Loss: 0.7012


Evaluation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 947/947 [01:36<00:00,  9.81it/s]


Epoch 5, Validation Loss: 1.0743
Epoch 5, Validation MSE Loss: 0.3739
Epoch 5, Validation BCE Loss: 0.7004
Epoch 5, MSE Hours: 0.7226
Epoch 5, MSE Minutes: 0.3974


Entraînement:   9%|██████████████                                                                                                                                           | 347/3785 [00:37<06:14,  9.18it/s]


KeyboardInterrupt: 

Sur les cinq premiers epochs, on observe que les pertes d'entrainement et de validation sont extrêmement stables, et ce, dès le premier epoch. L'interprétation est assez claire : le modèle n'apprend plus. **On arrête donc l'entraînement pendant le sixième epoch.**

On enregistre dans un fichier checkpoint les valeurs de normalisation, pour pouvoir recalculer facilement les sorties du modèle en minutes et heures réelles.

In [17]:
# Stockage des valeurs de normalisation et du nombre de sorties du modèle pour pouvoir le recharger
normalization_values = {
    "quantiles_hours": quantiles_hours.to_dict(),
    "quantiles_minutes": quantiles_minutes.to_dict(),
    "iqr_hours": iqr_hours,
    "iqr_minutes": iqr_minutes,
    "num_outputs": y_train.shape[1]
}

import json
with open("checkpoint/normalization.json", "w") as f:
    json.dump(normalization_values, f)